# Guia para a aula experimental

Nesse guia trataremos passo a passo a conexão de um dispositivo com a plataforma Konker.
Para seguir esse roteiro, você precisará de uma conta na Konker, que pode ser criada em https://demo.konkerlabs.net/ e, caso deseje trabalhar com o kit disponibilizado para vocês, será necessário baixar a Arduino IDE e adicionar o suporte a placa de desenvolvimento **NodeMCU**, além de instalar as bibliotecas **PubSubClient** e **ArduinoJSON**. Um guia de instalação da placa pode ser visto na referência https://www.filipeflop.com/blog/programar-nodemcu-com-ide-arduino/ enquanto a instalação das bibliotecas pode ser feita sem dificuldades buscando pelo nome das mesmas na caixa aberta ao selecionar o menu **Sketch** -> **Incluir Biblioteca** -> **Gerenciar Bibliotecas**.

Vamos iniciar nosso notebook chamando todas as bibliotecas que usaremos. Nessa etapa, nada precisa ser modificado.

In [ ]:
!pip install arrow bokeh paho.mqtt
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import pprint
import numpy as np
import arrow
import requests                                                                                                                                                                                 
import json                                                                                                                                                                                     
from threading import Timer

O próximo passo é definir os endereços que serão usados para consultar e enviar os dados. Nessa etapa ainda não é necessário fazer modificações.

In [ ]:
#Url de publicacao dos dados
pub_url = 'https://data.demo.konkerlabs.net/pub/'
#Url da API
base_api = 'https://api.demo.konkerlabs.net'
#Application padrão
application = 'default'

Agora vamos colocar em duas variáveis o seu usuário e senha da plataforma:

In [ ]:
username = ''
password = ''

## Usando o Kit de Hardware 
Esse é o momento de você baixar o código dos dispositivos disponível em nosso GitHub: https://github.com/KonkerLabs/arduino_examples. Você deve baixar o código **Termometro_MQTT**. Depois de baixar o código, abra o código do Termometro na Arduino IDE e mude os parâmetros de rede Wifi, canal de publicação (minha sugestão é usar *temperatura*) credenciais do dispositivo.

![term](https://raw.githubusercontent.com/KonkerLabs/arduino_examples/master/Termometro_MQTT/term.jpg "Termômetro")

Com o dispositivo montado, o próximo passo é compilar e gravar o Firmware. Lembre-se de mudar a board na Arduino IDE para **NodeMCU v1.0**.

Caso tudo tenha dado certo até o momento, você começará a observar os dados sendo enviados para a plataforma. Entre na Guia de Devices e procure *Messages* do seu dispositivo termômetro. Você deve ver os dados de temperatura chegando.

**Aguarde ao menos 1 minuto registrando a temperatura ambiente. Após isso, segure o termistor com seus dedos fazendo a temperatura subir. Mantenha o termômetro aquecido com seus dedos por ao menos 1 minuto também. Por último, deixe o dispositivo capturando novamente dados de temperatura ambiente enquanto roda o restante do notebook.**

## Usando a API da Konker para obter os dados e analisa-los localmente
Para iniciar esse trabalho, vamos primeiro conectar na API da Konker. A API usa OAuth2, então primeiro vamos obter as credenciais.

In [ ]:
client = BackendApplicationClient(client_id=username)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url='{}/v1/oauth/token'.format(base_api),
                                       client_id=username,
                                       client_secret=password)

Ótimo! Agora nos podemos começar a usar a API. Caso você queira mais detalhes de sua utilização, pode encontrar documentação em: https://api.demo.konkerlabs.net .

Vamos começar listando os dispositivos registrados no seu usuário.

In [ ]:
devices = oauth.get("https://api.demo.konkerlabs.net/v1/{}/devices/".format(application)).json()['result']
for dev in devices:
    print(dev)

Vamos procurar pelo dispositivo Termometro na sua lista de dispositivos:

In [ ]:
guid_term=""
for dev in devices:
    if dev['name'] == termometro_name:
        guid_term = dev['guid']

print("O GUID do dispositivo Termômetro é: "+ guid_term)

Caso você consiga ver o GUID do dispositivo, significa que está tudo funcionando bem. Caso o GUID não apareça, revise o nome do dispositivo no Notebook e o nome escolhido na plataforma para garantir que eles possuem a mesma grafia.

Agora vamos usar a API para pegar os dados enviados pelo dispositivo termômetro hoje. Caso você tenha escolhido outro canal para envio dos dados, por favor modifique a variável **canal** na próxima célula.

In [ ]:
canal = 'temperatura'
dt_start = arrow.utcnow().to('America/Sao_Paulo').floor('day')
dt_start = dt_start.shift(days=-1)
stats = oauth.get("https://api.demo.konkerlabs.net/v1/{}/incomingEvents?q=device:{} channel:{} timestamp:>{}&sort=oldest&limit=10000".format(application,guid_term,canal,dt_start.isoformat())).json()['result']
print(stats)

Caso tudo tenha funcionado como esperado, você deve estar vendo seus dados de temperatura logo acima. Para facilitar a visualização e análise dos dados, podemos transformar em um formato tabular com o Pandas.

In [ ]:
from pandas.io.json import json_normalize
stats_df = json_normalize(stats).set_index('timestamp')
stats_df = stats_df[3:]
stats_df

Ótimo! Agora os dados estão em um formato mais fácil de ler. Mas podemos também fazer um gráfico bem simples!

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(15,4))

stats_df['payload.value'].plot();

Agora começa a parte final desse trabalho. Vamos rodar um algoritmo conhecido com KMeans de aprendizado não supervisionado tentando encontrar os dois clusters que melhor separam nosso dataset. Como você pode observar abaixo, estamos usando a biblioteca SKLearn do Python para isso.

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2)
kmeans.fit(stats_df['payload.value'].values.reshape(-1, 1))

Nós colocamos como condição a separação em 2 clusters. Vamos ver qual a previsão feita sobre os dados adquiridos:

In [ ]:
predictions = kmeans.predict(stats_df['payload.value'].values.reshape(-1, 1))
predictions

Vamos observar a temperatura média do primeiro cluster de dados (grupo 0):

In [ ]:
print('Temperatura média do grupo 0: ' + str(stats_df.loc[predictions == 0]['payload.value'].mean()) + ' C')

E agora a temperatura média do segundo cluster (grupo 1):

In [ ]:
print('Temperatura média do grupo 1: ' + str(stats_df.loc[predictions == 1]['payload.value'].mean()) + ' C')

Vamos ver agora como se comportam os dois clusters encontrados em um gráfico. 

Nota: Nessa segunda versão do Notebook estou usando a biblioteca Bokeh para tentar evitar o gargalo em processamento gerado pelo Matplotlib na sala de aula. Caso você ainda encontre problemas em gerar o gráfico, por favor me envie um email em luis@konkerlabs.com

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import pandas as pd
output_notebook()

In [ ]:
p = figure(plot_width=820, plot_height=400, x_axis_type="datetime",
           title="Clusters de temperatura encontrados pelo método KMeans",
           x_axis_label='Tempo', 
           y_axis_label='Temperatura [Celsius]')

p.title.text_font_size = '18pt'
p.xaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label_text_font_size = "14pt"

x = np.array(pd.to_datetime(stats_df.index))
y = np.array(stats_df['payload.value'])
n_y = np.multiply(np.array(stats_df['payload.value']),predictions)
n_y = np.clip(n_y,np.min(y),np.max(y))

# add a line renderer
p.line(x, y, line_width=2)
p.patch(x,n_y,color="red",alpha = 0.5,line_width=0)

show(p) # show the results

Observe que em momento algum você precisou escolher um threshold para a divisão entre os clusters de temperatura. Muito embora esses dados representem coisas muito bem conhecidas, elas poderiam representar a situação normal de trabalho de uma máquina e a situação de superaquecimento.

Tipicamente em um sistema de IoT em produção, essa função por uma aplicação rodando na nuvem. A ideia aqui é apenas ilustrar esse processo usando ferramentas mais simples.
